<a href="https://colab.research.google.com/github/adepril/Analyse-des-lancements-spaciaux/blob/main/SpaceLaunches.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Analyse des lancements spatiaux (jeu de données récupéré sur Kaggle). 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

df = pd.read_csv('https://raw.githubusercontent.com/adepril/datasets/main/space_launches.csv')
df.head()

,Unnamed: 0,Unnamed: 0.1,Company Name,Location,Datum,Detail,Status Rocket,Rocket,Status Mission
0,0,0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA","Fri Aug 07, 2020 05:12 UTC",Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.0,Success
1,1,1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...","Thu Aug 06, 2020 04:01 UTC",Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success
2,2,2,SpaceX,"Pad A, Boca Chica, Texas, USA","Tue Aug 04, 2020 23:57 UTC",Starship Prototype | 150 Meter Hop,StatusActive,NaN,Success
3,3,3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan","Thu Jul 30, 2020 21:25 UTC",Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.0,Success
4,4,4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA","Thu Jul 30, 2020 11:50 UTC",Atlas V 541 | Perseverance,StatusActive,145.0,Success


Dans un premier temps, nous allons supprimer les deux colonne "Unnamed 0" et "Unnamed 0.1" qui ne servent à rien.

In [2]:
df = df.drop(['Unnamed: 0','Unnamed: 0.1'],axis=1).reset_index(drop=True)
df.sample(5)

,Company Name,Location,Datum,Detail,Status Rocket,Rocket,Status Mission
4081,US Air Force,"SLC-5, Vandenberg AFB, California, USA","Sun Jun 16, 1963 01:49 UTC",Scout X-3 | Transit-5A 3,StatusRetired,NaN,Success
2273,RVSN USSR,"Site 132/2, Plesetsk Cosmodrome, Russia","Tue Apr 19, 1983 12:00 UTC",Cosmos-3M (11K65M) | Cosmos 1453,StatusRetired,NaN,Success
42,VKS RF,"Site 43/4, Plesetsk Cosmodrome, Russia","Mon Mar 16, 2020 18:28 UTC",Soyuz 2.1b/Fregat-M | Cosmos 2545,StatusActive,NaN,Success
3719,RVSN USSR,"Site 41/1, Plesetsk Cosmodrome, Russia","Fri Nov 03, 1967 11:20 UTC",Voskhod | Cosmos 190,StatusRetired,NaN,Success
4171,RVSN USSR,"Mayak-2, Kapustin Yar, Russia","Fri Apr 06, 1962 17:15 UTC",Cosmos-2I (63S1) | Cosmos 2,StatusRetired,NaN,Success


Création des nouvelles colonnes : "Country" , "LaunchTime et LaunchDate : à partir de Datum pour une manipulation plus facile de l'heure et de la date



In [3]:
#Country: à partir de "Location"
df['country'] = [str(val).split(',')[-1].lstrip() for val in df.loc[:]['Location']]
#-Launch_time et Launch_date : à partir de Datum pour une manipulation plus facile de l'heure et de la date
df['launch_time']=df['Datum'].str.split(" ").str[-2]
df['launch_date']=df['Datum'].str.split(", ").str[-2]
#creation colonne "Year"
df['launch_year']=df['Datum'].str.split(" ").str[-3]
df.head()
#df.sample(5)

,Company Name,Location,Datum,Detail,Status Rocket,Rocket,Status Mission,country,launch_time,launch_date,launch_year
0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA","Fri Aug 07, 2020 05:12 UTC",Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.0,Success,USA,05:12,Fri Aug 07,2020
1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...","Thu Aug 06, 2020 04:01 UTC",Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success,China,04:01,Thu Aug 06,2020
2,SpaceX,"Pad A, Boca Chica, Texas, USA","Tue Aug 04, 2020 23:57 UTC",Starship Prototype | 150 Meter Hop,StatusActive,NaN,Success,USA,23:57,Tue Aug 04,2020
3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan","Thu Jul 30, 2020 21:25 UTC",Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.0,Success,Kazakhstan,21:25,Thu Jul 30,2020
4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA","Thu Jul 30, 2020 11:50 UTC",Atlas V 541 | Perseverance,StatusActive,145.0,Success,USA,11:50,Thu Jul 30,2020


Création d'un dictionary mlappant les Compagnies au pays ...

In [4]:
dict_comp = dict(zip(df['Company Name'], df['country']))

... puis création d'une colonne "sat_country"

In [5]:
#Declare a Dictionary dict_comp to map company name to origin country
df['sat_country'] = df['Company Name'].map(dict_comp)
#df.sample(5)

Manipulation de la colonne "Status Mission" pour garder les valeurs "Success" et "Failure" (transformer "Partial Failure" et Prelaunch Failure" en "Failure").

In [6]:
# Il faut transformer "Partial Failure" et Prelaunch Failure" en "Failure"
df["Status Mission"].value_counts()

Success              3879
Failure               339
Partial Failure       102
Prelaunch Failure       4
Name: Status Mission, dtype: int64

In [7]:
# Nouvelle colonne "StatusMission" où dles valeurs "Partial Failure" et "Prelaunch Failure" sont transformées en "Failure"
df["status_mission"] = np.where(df["Status Mission"]=="Success","Success","Failure")
#Nouvelle colonne "Success" où success=1 et failure=0
df["success"] = np.where(df["Status Mission"]=="Success",1,0)
#Exemple
df.iloc[973]

Company Name                                      VKS RF
Location          Site 32/2, Plesetsk Cosmodrome, Russia
Datum                         Fri Dec 24, 2004 12:00 UTC
Detail                    Tsyklon-3 | Sich 1M & Micron 1
Status Rocket                              StatusRetired
 Rocket                                              NaN
Status Mission                           Partial Failure
country                                           Russia
launch_time                                        12:00
launch_date                                   Fri Dec 24
launch_year                                         2004
sat_country                                       Russia
status_mission                                   Failure
success                                                0
Name: 973, dtype: object

Manipulation des données en vue d'une représentation graphique

In [19]:
data_gp =df.groupby(["sat_country","launch_year"]).agg({"country":"count","success":"sum"}).reset_index()
data_gp.head(3)

,sat_country,launch_year,country,success
0,Australia,1968,1,1
1,Australia,1969,1,0
2,Australia,1970,2,0


In [ ]:
df['total_launch'] = data_gp['country']
df['total_launch'].value_counts()

In [21]:
data_gp.rename(columns={"country":"total_launch"},inplace=True)
data_gp.head(3)

,sat_country,launch_year,total_launch,success
0,Australia,1968,1,1
1,Australia,1969,1,0
2,Australia,1970,2,0


In [22]:
idx = pd.MultiIndex.from_product([data_gp['launch_year'].unique(), data_gp['sat_country'].unique()], names=['launch_year', 'sat_country']) 
#print(idx)

In [23]:
dat2 = data_gp.set_index(['launch_year', 'sat_country']).reindex(idx).fillna(0).sort_values(ascending=True,by=["sat_country","launch_year"]) 
#print(dat2)

In [24]:
dat2 = pd.concat([dat2, dat2.groupby(level=1).cumsum().add_prefix('cum_')], 1).sort_index(level=1).reset_index() 
#dat2

In [25]:
dat2.sort_values(ascending=[True,False],by=["launch_year","sat_country"],inplace=True)
#dat2

In [26]:
dat2.rename(columns={"launch_year":"Year","cum_total_launch":"Cumulative Launches","sat_country":"Country","cum_success":"Cumulative Successes"},inplace=True) 
#dat2

#Animation


In [ ]:
px.bar(dat2,x="Country",y="Cumulative Launches",color="Country",animation_group="Country",animation_frame="Year",range_y=[0,2200],width=1200)

Succès cumulés / Lancements cumulés

In [ ]:
fig=px.scatter(
    dat2, x="Cumulative Launches", y="Cumulative Successes", animation_frame="Year", animation_group="Country",
    color="Country", hover_name="Country",size="Cumulative Launches", size_max=55, range_x=[0,2400], range_y=[0,2200],
    width=1000
    )
fig.update_layout(showlegend=True)

TODO


In [30]:
#remplace "NaN" par "0" dans la colonne "total_launch"
df['total_launch'].fillna(0, inplace = True)
df.sample(5)
df[df["sat_country"]=="USA"]

,Company Name,Location,Datum,Detail,Status Rocket,Rocket,Status Mission,country,launch_time,launch_date,launch_year,sat_country,status_mission,success,total_launch
0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA","Fri Aug 07, 2020 05:12 UTC",Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.0,Success,USA,05:12,Fri Aug 07,2020,USA,Success,1,1.0
2,SpaceX,"Pad A, Boca Chica, Texas, USA","Tue Aug 04, 2020 23:57 UTC",Starship Prototype | 150 Meter Hop,StatusActive,NaN,Success,USA,23:57,Tue Aug 04,2020,USA,Success,1,2.0
4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA","Thu Jul 30, 2020 11:50 UTC",Atlas V 541 | Perseverance,StatusActive,145.0,Success,USA,11:50,Thu Jul 30,2020,USA,Success,1,1.0
8,SpaceX,"SLC-40, Cape Canaveral AFS, Florida, USA","Mon Jul 20, 2020 21:30 UTC",Falcon 9 Block 5 | ANASIS-II,StatusActive,50.0,Success,USA,21:30,Mon Jul 20,2020,USA,Success,1,1.0
10,Northrop,"LP-0B, Wallops Flight Facility, Virginia, USA","Wed Jul 15, 2020 13:46 UTC",Minotaur IV | NROL-129,StatusActive,46.0,Success,USA,13:46,Wed Jul 15,2020,USA,Success,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4317,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA","Mon Mar 17, 1958 12:15 UTC",Vanguard | Vanguard 1,StatusRetired,NaN,Success,USA,12:15,Mon Mar 17,1958,USA,Success,1,0.0
4318,AMBA,"LC-26A, Cape Canaveral AFS, Florida, USA","Wed Mar 05, 1958 18:27 UTC",Juno I | Explorer 2,StatusRetired,NaN,Failure,USA,18:27,Wed Mar 05,1958,USA,Failure,0,0.0
4319,US Navy,"LC-18A, Cape Canaveral AFS, Florida, USA","Wed Feb 05, 1958 07:33 UTC",Vanguard | Vanguard TV3BU,StatusRetired,NaN,Failure,USA,07:33,Wed Feb 05,1958,USA,Failure,0,0.0
4320,AMBA,"LC-26A, Cape Canaveral AFS, Florida, USA","Sat Feb 01, 1958 03:48 UTC",Juno I | Explorer 1,StatusRetired,NaN,Success,USA,03:48,Sat Feb 01,1958,USA,Success,1,0.0
